In [1]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import cv2
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import random
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Dropout, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2

In [31]:
trainGen = ImageDataGenerator(
rescale=1/255.,
horizontal_flip=True,
validation_split = 0.1
)
testGen =ImageDataGenerator(
rescale= 1/255.,
)



In [34]:
train = trainGen.flow_from_directory(
    'D:/DATASETS/datasets/FaceMaskDataset/train/', 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='training'
)

validation = trainGen.flow_from_directory(
    'D:/DATASETS/datasets/FaceMaskDataset/test/', 
    target_size=(224, 224),
    classes=['with_mask','without_mask'],
    class_mode='categorical', 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='validation'
)



Found 2268 images belonging to 2 classes.
Found 128 images belonging to 2 classes.


In [35]:
mob = MobileNetV2(
    input_shape = (224,224,3),
    include_top = False,
    weights = 'imagenet',
)
mob.trainable = False

In [37]:
maskmodel = Sequential()
maskmodel.add(mob)
maskmodel.add(GlobalAveragePooling2D())
maskmodel.add(Dense(64,activation='relu'))
maskmodel.add(Dropout(0.3))
maskmodel.add(Dense(2,activation='softmax'))
maskmodel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                81984     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 130       
Total params: 2,340,098
Trainable params: 82,114
Non-trainable params: 2,257,984
_________________________________________________________________


In [39]:
maskmodel.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['acc'])

In [41]:
checkpoint = ModelCheckpoint(
    'model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min'
)
hist = maskmodel.fit(
    train,
    epochs = 15,
    validation_data = validation,
    callbacks = [checkpoint]
)

Epoch 1/15
64/71 [==========================>...] - ETA: 4s - loss: 0.0253 - acc: 0.9922

D:\Anaconda3\lib\site-packages\PIL\Image.py:975: UserWarning:

Palette images with Transparency expressed in bytes should be converted to RGBA images



71/71 [==============================] - ETA: 0s - loss: 0.0249 - acc: 0.9921
Epoch 00001: val_loss improved from inf to 0.12918, saving model to model.h5
71/71 [==============================] - 58s 816ms/step - loss: 0.0249 - acc: 0.9921 - val_loss: 0.1292 - val_acc: 0.9531
Epoch 2/15
71/71 [==============================] - ETA: 0s - loss: 0.0101 - acc: 0.9974
Epoch 00002: val_loss did not improve from 0.12918
71/71 [==============================] - 55s 769ms/step - loss: 0.0101 - acc: 0.9974 - val_loss: 0.3218 - val_acc: 0.8906
Epoch 3/15
71/71 [==============================] - ETA: 0s - loss: 0.0121 - acc: 0.9965
Epoch 00003: val_loss did not improve from 0.12918
71/71 [==============================] - 52s 727ms/step - loss: 0.0121 - acc: 0.9965 - val_loss: 0.1484 - val_acc: 0.9531
Epoch 4/15
71/71 [==============================] - ETA: 0s - loss: 0.0036 - acc: 0.9987
Epoch 00004: val_loss improved from 0.12918 to 0.12314, saving model to model.h5
71/71 [======================

In [42]:
maskmodel.save('mobilenet_model.h5')

In [44]:
maskmodel.evaluate(validation,verbose=0)

[0.24498730897903442, 0.9296875]

In [2]:
#IMPLEMENTING LIVE DETECTION OF FACE MASK
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow import keras
import tensorflow.keras.backend as k
from tensorflow.keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import cv2
import datetime

from tensorflow.keras.preprocessing import image
import cv2
mymodel = load_model('mobilenet_model.h5')

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('D:/haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()